In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from sklearn.model_selection import train_test_split

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [11]:
# Load the dataset
df = pd.read_csv('edited_data.csv')

In [13]:
df.head()

,Chapter No,Verse No,Shloka,English Translation,Explanation
0,1,1,धृतराष्ट्र उवाच | धर्मक्षेत्रे कुरुक्षेत्रे सम...,"Dhritarastra said: O Sanjaya, what did my sons...",The two armies had gathered on the battlefield...
1,1,2,सञ्जय उवाच । दृष्ट्वा तु पाण्डवानीकं व्यूढं दु...,"Sanjaya said: But then, seeing the army of the...","Sanjay understood Dhritarashtra’s concern, who..."
2,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् । व...,"O teacher, (please) see this vast army of the ...",Duryodhana asked Dronacharya to look at the sk...
3,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि | युयुधा...,"There are in this army, heroes wielding great ...","Due to his anxiety, the Pandava army seemed mu..."
4,1,5,धृष्टकेतुश्चेकितान: काशिराजश्च वीर्यवान् | पुर...,"Dhrstaketu, Cekitana, and the valiant king of ...","Due to his anxiety, the Pandava army seemed mu..."


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Chapter No           700 non-null    int64 
 1   Verse No             700 non-null    int64 
 2   Shloka               700 non-null    object
 3   English Translation  700 non-null    object
 4   Explanation          700 non-null    object
dtypes: int64(2), object(3)
memory usage: 27.5+ KB


In [15]:
max_length = max([len(verse.split()) for verse in df['Shloka']])
num_classes = df['Chapter No'].nunique()

In [19]:
X = df['Shloka']
y = pd.get_dummies(df['Chapter No']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
vocab_size = len(tokenizer.word_index) + 1

In [24]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=128))
model.add(Dense(units=num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# Convert text to sequences of integers
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length
X_train_seq_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')


In [26]:
#After preprocessing the data, you can train the model using the fit() method
model.fit(X_train_seq_padded, y_train, validation_data=(X_test_seq_padded, y_test), epochs=10, batch_size=64)


Epoch 1/10
9/9 [==============================] - 6s 162ms/step - loss: 2.8722 - accuracy: 0.0839 - val_loss: 2.8310 - val_accuracy: 0.1286
Epoch 2/10
9/9 [==============================] - 0s 44ms/step - loss: 2.8421 - accuracy: 0.1071 - val_loss: 2.8094 - val_accuracy: 0.1286
Epoch 3/10
9/9 [==============================] - 0s 42ms/step - loss: 2.8205 - accuracy: 0.1107 - val_loss: 2.8231 - val_accuracy: 0.1286
Epoch 4/10
9/9 [==============================] - 0s 41ms/step - loss: 2.8094 - accuracy: 0.1071 - val_loss: 2.8175 - val_accuracy: 0.1286
Epoch 5/10
9/9 [==============================] - 0s 41ms/step - loss: 2.7495 - accuracy: 0.1107 - val_loss: 2.7726 - val_accuracy: 0.1143
Epoch 6/10
9/9 [==============================] - 0s 41ms/step - loss: 2.3912 - accuracy: 0.1571 - val_loss: 3.2730 - val_accuracy: 0.1143
Epoch 7/10
9/9 [==============================] - 0s 41ms/step - loss: 2.3492 - accuracy: 0.1643 - val_loss: 3.4158 - val_accuracy: 0.0571
Epoch 8/10
9/9 [==========

In [27]:
#evaluation
loss, accuracy = model.evaluate(X_test_seq_padded, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


5/5 [==============================] - 0s 10ms/step - loss: 3.1884 - accuracy: 0.0857
Test loss: 3.188405990600586
Test accuracy: 0.08571428805589676


The test loss is 3.188 and the test accuracy is 0.086. This means that the model is not performing very well and needs improvement. The accuracy of 0.086 indicates that the model is correctly predicting the chapter number of only about 8.6% of the verses in the test set. 

Training for model's better perfomance 

In [29]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [42]:
df = pd.read_csv('edited_data.csv')
max_length = max([len(verse.split()) for verse in df['Shloka']])
num_classes = df['Chapter No'].nunique()
X = df['Shloka']
y = pd.get_dummies(df['Chapter No']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
vocab_size = len(tokenizer.word_index) + 1
# Try different embedding dimensions
embedding_dim = 300

# Use pre-trained word embeddings such as GloVe or Word2Vec
# embedding_matrix = create_embedding_matrix('path/to/embeddings_file', tokenizer.word_index, embedding_dim)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))

Changing the embedding dimension can help the model performance in a few ways. The embedding layer is responsible for learning the relationship between words in the input text, and the embedding dimension represents the size of the vector that represents each word. A higher embedding dimension can capture more complex relationships between words, but can also increase the number of parameters in the model, making it more computationally expensive and potentially leading to overfitting.

In some cases, a higher embedding dimension may lead to better performance, as it allows the model to capture more nuanced relationships between words. However, the optimal embedding dimension can vary depending on the size of the dataset, the complexity of the task, and the specific language used in the input text. Therefore, it's important to experiment with different embedding dimensions to find the one that works best for a particular task.

In [43]:
# Add more dense layers after the LSTM layer to increase model capacity
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=num_classes, activation='softmax'))

In [44]:
# Use regularization techniques such as dropout to prevent overfitting
# model.add(Dropout(0.2))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert text to sequences of integers
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [45]:
# Pad sequences to the same length
X_train_seq_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Increase the number of epochs to allow the model to learn more from the training data
model.fit(X_train_seq_padded, y_train, validation_data=(X_test_seq_padded, y_test), epochs=50, batch_size=64)


Epoch 1/50
9/9 [==============================] - 10s 289ms/step - loss: 2.8781 - accuracy: 0.0750 - val_loss: 2.8533 - val_accuracy: 0.0500
Epoch 2/50
9/9 [==============================] - 1s 157ms/step - loss: 2.8503 - accuracy: 0.0732 - val_loss: 2.8323 - val_accuracy: 0.1643
Epoch 3/50
9/9 [==============================] - 1s 145ms/step - loss: 2.8330 - accuracy: 0.0929 - val_loss: 2.8313 - val_accuracy: 0.1286
Epoch 4/50
9/9 [==============================] - 1s 135ms/step - loss: 2.8025 - accuracy: 0.1161 - val_loss: 2.7884 - val_accuracy: 0.1357
Epoch 5/50
9/9 [==============================] - 1s 139ms/step - loss: 2.5330 - accuracy: 0.1429 - val_loss: 3.1116 - val_accuracy: 0.1000
Epoch 6/50
9/9 [==============================] - 1s 139ms/step - loss: 2.5129 - accuracy: 0.1286 - val_loss: 3.0574 - val_accuracy: 0.0929
Epoch 7/50
9/9 [==============================] - 1s 135ms/step - loss: 2.5288 - accuracy: 0.1179 - val_loss: 2.9331 - val_accuracy: 0.0500
Epoch 8/50
9/9 [===

In [46]:
# evaluation
loss, accuracy = model.evaluate(X_test_seq_padded, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

5/5 [==============================] - 0s 41ms/step - loss: 9.1079 - accuracy: 0.1143
Test loss: 9.107884407043457
Test accuracy: 0.11428571492433548


Overfitting occured - 

In [41]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional, Conv1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from gensim.models import KeyedVectors

# Load pre-trained word embeddings
word_vectors = KeyedVectors.load_word2vec_format('path/to/embedding/file', binary=True)

df = pd.read_csv('edited_data.csv')

max_length = max([len(verse.split()) for verse in df['Shloka']])
num_classes = df['Chapter No'].nunique()

X = df['Shloka']
y = pd.get_dummies(df['Chapter No']).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 300

# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word_vectors.vocab:
        embedding_matrix[i] = word_vectors[word]

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))

# Add multiple LSTM layers and a dropout layer
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128))

# Add dense layers with different activation functions
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile model with different optimizer and learning rate
from keras.optimizers import Adam
opt = Adam(learning_rate=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Increase batch size and number of epochs
model.fit(X_train_seq_padded, y_train, validation_data=(X_test_seq_padded, y_test), epochs=20, batch_size=128)

# Add a bidirectional LSTM layer and a convolutional layer
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile model with different optimizer and learning rate
from keras.optimizers import SGD
opt = SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Increase batch size and number of epochs
model.fit(X_train_seq_padded, y_train, validation_data=(X_test_seq_padded, y_test), epochs=30, batch_size=256)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_seq_padded, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


ModuleNotFoundError: No module named 'gensim'